In [8]:
import numpy as np
from sklearn.neighbors import BallTree
import time

np.random.seed(42)

n_samples = 10_000

# Random locations on Earth
latitudes = np.random.uniform(-90, 90, n_samples)
longitudes = np.random.uniform(-180, 180, n_samples)

# Stack into (lat, lon)
entregas_deg = np.column_stack([latitudes, longitudes])

#BallTree with haversine requires radians.
entregas_rad = np.radians(entregas_deg)

In [9]:
from sklearn.neighbors import BallTree

tree = BallTree(entregas_rad, metric='haversine', leaf_size=40)


In [10]:
n_cds = 50

# Random locations on Earth
lat_cds = np.random.uniform(-90, 90, n_cds)
long_cds = np.random.uniform(-180, 180, n_cds)

# Stack into (lat, lon)
coord_cds = np.column_stack([lat_cds, long_cds])

In [11]:
k = 1 # Number of nearest neighbors to find

start = time.time()
dist_bt, ind_bt = tree.query(coord_cds, k=k)
balltree_query_time = time.time() - start

print(f"Ball Tree query time: {balltree_query_time:.4f} s")


Ball Tree query time: 0.0050 s


In [12]:
#Comparacao com brute-force
EARTH_RADIUS_KM = 6371.0

def haversine_vec(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * np.arcsin(np.sqrt(a))

start = time.time()

for q in coord_cds:
    haversine_vec(q[0], q[1], entregas_deg[:, 0], entregas_deg[:, 1])

bruteforce_time = time.time() - start

#Nota: aqui só faz as multiplicações, não pega os menores

print(f"Brute-force query time: {bruteforce_time:.4f} s")

Brute-force query time: 0.0487 s


In [13]:
query_sp_deg = np.array([[-23.5505, -46.6333]])
query_sp_rad = np.radians(query_sp_deg)

dist_sp_rad, ind_sp = tree.query(query_sp_rad, k=5)
dist_sp_km = dist_sp_rad[0] * EARTH_RADIUS_KM
neighbors = entregas_deg[ind_sp[0]]

print("\nNearest neighbors to São Paulo:")
for i, (pt, d) in enumerate(zip(neighbors, dist_sp_km), 1):
    print(f"{i}: lat={pt[0]:.4f}, lon={pt[1]:.4f}, dist={d:.2f} km")



Nearest neighbors to São Paulo:
1: lat=-22.5233, lon=-47.2023, dist=128.20 km
2: lat=-22.5828, lon=-45.4893, dist=158.98 km
3: lat=-22.9958, lon=-48.1020, dist=162.20 km
4: lat=-23.0252, lon=-49.0370, dist=252.36 km
5: lat=-21.0212, lon=-45.9119, dist=290.87 km
